<a href="https://www.kaggle.com/code/mateotfuentes/xgboost-for-disaster-tweets?scriptVersionId=106484721" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [2]:
train_data = pd.read_csv('../input/nlp-getting-started/train.csv')

In [3]:
print(train_data)

         id keyword location  \
0         1     NaN      NaN   
1         4     NaN      NaN   
2         5     NaN      NaN   
3         6     NaN      NaN   
4         7     NaN      NaN   
...     ...     ...      ...   
7608  10869     NaN      NaN   
7609  10870     NaN      NaN   
7610  10871     NaN      NaN   
7611  10872     NaN      NaN   
7612  10873     NaN      NaN   

                                                   text  target  
0     Our Deeds are the Reason of this #earthquake M...       1  
1                Forest fire near La Ronge Sask. Canada       1  
2     All residents asked to 'shelter in place' are ...       1  
3     13,000 people receive #wildfires evacuation or...       1  
4     Just got sent this photo from Ruby #Alaska as ...       1  
...                                                 ...     ...  
7608  Two giant cranes holding a bridge collapse int...       1  
7609  @aria_ahrary @TheTawniest The out of control w...       1  
7610  M1.94 [01:04 UT

In [4]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [5]:
train_data.keyword = train_data.keyword.fillna(train_data.keyword.mode()[0])

In [6]:
train_data

,id,keyword,location,text,target
0,1,fatalities,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,fatalities,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,fatalities,NaN,All residents asked to 'shelter in place' are ...,1
3,6,fatalities,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,fatalities,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,fatalities,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,fatalities,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,fatalities,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,fatalities,NaN,Police investigating after an e-bike collided ...,1


In [7]:
train_data.keyword.unique()

array(['fatalities', 'ablaze', 'accident', 'aftershock',
       'airplane%20accident', 'ambulance', 'annihilated', 'annihilation',
       'apocalypse', 'armageddon', 'army', 'arson', 'arsonist', 'attack',
       'attacked', 'avalanche', 'battle', 'bioterror', 'bioterrorism',
       'blaze', 'blazing', 'bleeding', 'blew%20up', 'blight', 'blizzard',
       'blood', 'bloody', 'blown%20up', 'body%20bag', 'body%20bagging',
       'body%20bags', 'bomb', 'bombed', 'bombing', 'bridge%20collapse',
       'buildings%20burning', 'buildings%20on%20fire', 'burned',
       'burning', 'burning%20buildings', 'bush%20fires', 'casualties',
       'casualty', 'catastrophe', 'catastrophic', 'chemical%20emergency',
       'cliff%20fall', 'collapse', 'collapsed', 'collide', 'collided',
       'collision', 'crash', 'crashed', 'crush', 'crushed', 'curfew',
       'cyclone', 'damage', 'danger', 'dead', 'death', 'deaths', 'debris',
       'deluge', 'deluged', 'demolish', 'demolished', 'demolition',
       'dera

Los "%20" significan espacios así que vamos a cambiarlo. This way it will allow for relations in words inside those two words separated by %20. I mean, if we split airplane%accident into airplane accident it will recognize the presence of "accident"

In [8]:
train_data.keyword = [doc.replace ("%20", " ") for doc in train_data.keyword]

In [9]:
train_data.keyword.unique()

array(['fatalities', 'ablaze', 'accident', 'aftershock',
       'airplane accident', 'ambulance', 'annihilated', 'annihilation',
       'apocalypse', 'armageddon', 'army', 'arson', 'arsonist', 'attack',
       'attacked', 'avalanche', 'battle', 'bioterror', 'bioterrorism',
       'blaze', 'blazing', 'bleeding', 'blew up', 'blight', 'blizzard',
       'blood', 'bloody', 'blown up', 'body bag', 'body bagging',
       'body bags', 'bomb', 'bombed', 'bombing', 'bridge collapse',
       'buildings burning', 'buildings on fire', 'burned', 'burning',
       'burning buildings', 'bush fires', 'casualties', 'casualty',
       'catastrophe', 'catastrophic', 'chemical emergency', 'cliff fall',
       'collapse', 'collapsed', 'collide', 'collided', 'collision',
       'crash', 'crashed', 'crush', 'crushed', 'curfew', 'cyclone',
       'damage', 'danger', 'dead', 'death', 'deaths', 'debris', 'deluge',
       'deluged', 'demolish', 'demolished', 'demolition', 'derail',
       'derailed', 'derailment

In [10]:
from sklearn.feature_extraction.text import CountVectorizer 
vect = CountVectorizer() 

In [11]:
test_data = pd.read_csv('../input/nlp-getting-started/test.csv') 

In [12]:
test_data

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [13]:
train_texts = train_data.text
test_texts = test_data.text

In [14]:
total_texts = np.concatenate([train_texts, test_texts])

In [15]:
vect.fit(total_texts)

CountVectorizer()

In [16]:
train_bow = vect.transform(train_data.text) 
test_bow  = vect.transform(test_data.text) 

In [17]:
X = train_bow
y = train_data.target 

In [18]:
from sklearn.model_selection import train_test_split
train_X, val_X, train_y, val_y = train_test_split(X, y, test_size = 0.2) 

We cannot use f1 as our objective function, as it is not differentiable. I will thus use log-loss, which is the default objective function of XGBClassifier 

 como ejercicio puedo intentar crear un plot con cada una de las learning rates y compararlas 

In [19]:
import sklearn
import sklearn.metrics

Now we are going to implement our model. We will use xgboost and we'll try to fiund

In [26]:
import xgboost as xgb 
import sklearn.metrics

def geterror(real, pred): 
    return sklearn.metrics.log_loss(real, pred) 

xgb_p = xgb.XGBClassifier(n_estimators = 4000, early_stopping_rounds = 50, learning_rate = 0.01)
xgb_p.fit(train_X, train_y, 
         eval_set = [(train_X, train_y), (val_X,val_y)], 
         verbose = 100) 
pprediction = xgb_p.predict(val_X) 
blr = 0.01
lrates = [0.02, 0.05, 0.1, 0.2, 0.25, 0.5]
t_error = geterror(pprediction, val_y) 


for rate in lrates: 
    print("Comienza un nuevo")
    xgb_p = xgb.XGBClassifier(n_estimators = 4000, early_stopping_rounds = 50, learning_rate = rate) 
    xgb_p.fit(train_X, train_y, 
             eval_set = [(train_X, train_y), (val_X, val_y)], 
             verbose = 100)
    pprediction = xgb_p.predict(val_X)
    error = geterror(pprediction, val_y) 
    if error< t_error: 
        t_error = error 
        blr = rate




[0]	validation_0-logloss:0.69128	validation_1-logloss:0.69160
[100]	validation_0-logloss:0.59389	validation_1-logloss:0.61343
[200]	validation_0-logloss:0.55692	validation_1-logloss:0.58259
[300]	validation_0-logloss:0.53410	validation_1-logloss:0.56453
[400]	validation_0-logloss:0.51806	validation_1-logloss:0.55302
[500]	validation_0-logloss:0.50529	validation_1-logloss:0.54405
[600]	validation_0-logloss:0.49407	validation_1-logloss:0.53673
[700]	validation_0-logloss:0.48428	validation_1-logloss:0.53027
[800]	validation_0-logloss:0.47542	validation_1-logloss:0.52470
[900]	validation_0-logloss:0.46742	validation_1-logloss:0.51989
[1000]	validation_0-logloss:0.46031	validation_1-logloss:0.51569
[1100]	validation_0-logloss:0.45365	validation_1-logloss:0.51164
[1200]	validation_0-logloss:0.44748	validation_1-logloss:0.50789
[1300]	validation_0-logloss:0.44173	validation_1-logloss:0.50436
[1400]	validation_0-logloss:0.43618	validation_1-logloss:0.50112
[1500]	validation_0-logloss:0.43094	v

In [27]:
blr

0.02

In [28]:
xgb_p = xgb.XGBClassifier(n_estimators = 4000, early_stopping_rounds = 50, learning_rate = blr) 
xgb_p.fit(train_X, train_y, 
             eval_set = [(train_X, train_y), (val_X, val_y)], 
             verbose = 100)

[0]	validation_0-logloss:0.68943	validation_1-logloss:0.69008
[100]	validation_0-logloss:0.55638	validation_1-logloss:0.58253
[200]	validation_0-logloss:0.51801	validation_1-logloss:0.55289
[300]	validation_0-logloss:0.49403	validation_1-logloss:0.53703
[400]	validation_0-logloss:0.47516	validation_1-logloss:0.52505
[500]	validation_0-logloss:0.45976	validation_1-logloss:0.51607
[600]	validation_0-logloss:0.44677	validation_1-logloss:0.50813
[700]	validation_0-logloss:0.43545	validation_1-logloss:0.50158
[800]	validation_0-logloss:0.42516	validation_1-logloss:0.49611
[900]	validation_0-logloss:0.41584	validation_1-logloss:0.49114
[1000]	validation_0-logloss:0.40752	validation_1-logloss:0.48726
[1100]	validation_0-logloss:0.39983	validation_1-logloss:0.48383
[1200]	validation_0-logloss:0.39260	validation_1-logloss:0.48035
[1300]	validation_0-logloss:0.38579	validation_1-logloss:0.47750
[1400]	validation_0-logloss:0.37964	validation_1-logloss:0.47479
[1500]	validation_0-logloss:0.37379	v

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=50, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.02, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=4000,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [29]:
prediction = xgb_p.predict(test_bow)

In [33]:
sample_submission = pd.read_csv('../input/nlp-getting-started/sample_submission.csv')

In [34]:
sample_submission.target = prediction

In [35]:
sample_submission.to_csv("submission-csv", index = False) 